[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/UNSW-COMP9418/Week05/blob/main/COMP9418_W05_Markov_Chains_and_Hidden_Markov_Models.ipynb)

# Markov Chains and Hidden Markov Models

**COMP9418 W05 Tutorial**

- Instructor: Gustavo Batista
- School of Computer Science and Engineering, UNSW Sydney 
- Notebook designed by Gustavo Batista and Jeremy Gillen
- Last Update 6th September 2022

In this week's tutorial, we will explore models that use the Markovian assumptions, in particular Markov chains and Hidden Markov Models. We will implement the forward and Viterbi algorithms and use them to gain intuition about the convergence of Markov chains, and the probabilitic queries these algorithms can answer. 

## Technical prerequisites

You will need certain packages installed to run this notebook.

If you are using ``conda``'s default
[full installation](https://conda.io/docs/install/full.html),
these requirements should all be satisfied already.

If you are using ``virtualenv`` or other native package management,
you may need to run these commands:

```python
pip3 install numpy matplotlib
```

To render a visualization of some graphical models, you also need to install Graphviz [download page](http://www.graphviz.org/download). We have already used this library in Tutorial 1, thus, you should have it installed. If you do not have it and use the conda installation, then use the command ```conda install python-graphviz```.

Once we have done all that, we
import some useful modules for later use.

In [ ]:
# Necessary libraries
import numpy as np
import matplotlib.pyplot as plt

from graphviz import Digraph
# combinatorics
from itertools import product, combinations

from DiscreteFactors import Factor
from Graph import Graph 
from BayesNet import BayesNet 

## Markov Chains

Let's start implementing the mini-forward algorithm for Markov chains. We will use this algorithm to calculate the probability of a sequence of events as well as testing the convergence of some chains.

Remember from the course slides that a Markov chain (as well as the Hidden Markov Model) is a Dynamic Bayesian Network (DBN). It means that this network "grows", i.e., we can add nodes as we iterate over time or space. We need to specify a notation to indicate time passage. Similarly to the slides, we use $t-1$ and $t$ appended to the variables names to designate the previous time and present time.

We can use the example from the slides to introduce the notation and write the first transition tables.

![](https://raw.githubusercontent.com/UNSW-COMP9418/Week05/main/img/weather.png)

To define a Markov chain, we need to set the outcomeSpace as well as initial state and the transition probabilities. We will write the first two and let the third one as an exercise.

In [ ]:
# possible outcomes, by variable
outcomeSpace = {
    'Weather':('sun','rain'),    
    'Weather_next':('sun','rain'),
}

# The start state, in this case let's assume we start in a sunny day
weatherStart = Factor(('Weather',), outcomeSpace)
weatherStart['sun'] = 1.0
weatherStart['rain'] = 0.0

### Exercise

Now, it is your turn. Define the transition probability table according to the figure above. We have created an initial table for you.

In [ ]:
weatherTransition = Factor(('Weather', 'Weather_next'), outcomeSpace)
weatherTransition['sun', 'sun'] = ...
weatherTransition['sun', 'rain'] = ...
weatherTransition['rain', 'sun'] = ...
weatherTransition['rain', 'rain'] = ...

We can double-check our work with the `graphviz` library. The source code bellow draws a state transition graph based on the `weatherTransition` table that you just defined. Compare the `graphviz` plot with the slide figure to confirm your state transition probabilities are correct.

In [ ]:
dot = Digraph(engine="neato", comment='Weather Markov chain')
dot.attr(overlap="false", splines="false")

pos = {
    'sun': '1,1!',
    'rain': '0,0!',    
}

for v in outcomeSpace['Weather']:
    dot.node(v, pos=pos[v])

for v in outcomeSpace['Weather']:
    for w in outcomeSpace['Weather']:
        dot.edge(v, w, str(weatherTransition[v,w]))

dot

We can now implement the mini-forward algoritm for Markov chains. Remember from the slides that the update rule for the forward simulation is the following:

$P(x_t) = \sum_{x_{t-1}}P(x_t | x_{t-1})P(x_{t-1})$

- $P(x_{t-1})$ is the previous state probability table.

- $P(x_t | x_{t-1})$ is the transition probability table such as the one you defined in the previous exercise.

- $P(x_t)$ is the current state probability table.

From this equation, it is clear we need two basic operations. One is the multiplication of factors (probability tables) that we implemented in a previous tutorial as a `joint` operation. The second one summing one variable out that we implemented as a `marginalize` operation.

### Exercise

Let's implement the mini-Forward algorithm for Markov chains. We start with the online version that makes a single update (one time step). We use the online version to implement the batch one that runs multiple time steps.

In [ ]:
 class MarkovModel():
    def __init__(self, start_state, transition, variable_remap):
        '''
        Takes 3 arguments:
        - start_state: a factor representing the start state. E.g. domain might be ('A', 'B', 'C')
        - transition: a factor that represents the transition probs. E.g. P('A_next', 'B_next', 'C_next' | 'A', 'B', 'C')
        - variable_remap: a dictionary that maps new variable names to old variable names,
                          to reset the state after transition. E.g. {'A_next':'A', 'B_next':'B', 'C_next':'C'}
        '''
        self.state = start_state
        self.transition = transition
        self.remap = variable_remap

    def forward(self):
        # get state vars (to be marginalized later)
        state_vars = self.state.domain
        # join with transition factor
        f = ... # TODO 1 line
        # marginalize out old state vars, leaving only new state vars
        ... # TODO 2 lines
        # remap variables to their original names
        f.domain = tuple(self.remap[var] for var in f.domain)
        self.state = f
        return self.state


##################
# Test code

variable_remap = {
    "Weather_next": "Weather"
}
mm = MarkovModel(weatherStart, weatherTransition, variable_remap)

print(mm.forward())

If your implementation is correct, you should have the following output:

```
╒═══════════╤══════╕
│ Weather   │   Pr │
╞═══════════╪══════╡
│ sun       │  0.9 │
├───────────┼──────┤
│ rain      │  0.1 │
╘═══════════╧══════╛
```

We can use the mini-forward online implementation to make a simple batch extension that calls the online subroutine a fixed number of times.

### Exercise

Implement the batch version of the mini-forward algorithm for Markov chains

In [ ]:
class MarkovModel(MarkovModel):
    def forwardBatch(self, n):
        ''' Do `n` steps, and return history list of states '''
        history = []
        for i in range(n):
            ... # TODO forward
            ... # TODO append current state to history
        return history


##################
# Test code

variable_remap = {
    "Weather_next": "Weather"
}
mm = MarkovModel(weatherStart, weatherTransition, variable_remap)

history = mm.forwardBatch(3)
for factor in history:
    print(factor)

If you implemented your code correctly, you should see the following output:

```
╒═══════════╤══════╕
│ Weather   │   Pr │
╞═══════════╪══════╡
│ sun       │  0.9 │
├───────────┼──────┤
│ rain      │  0.1 │
╘═══════════╧══════╛

╒═══════════╤══════╕
│ Weather   │   Pr │
╞═══════════╪══════╡
│ sun       │ 0.84 │
├───────────┼──────┤
│ rain      │ 0.16 │
╘═══════════╧══════╛

╒═══════════╤═══════╕
│ Weather   │    Pr │
╞═══════════╪═══════╡
│ sun       │ 0.804 │
├───────────┼───────┤
│ rain      │ 0.196 │
╘═══════════╧═══════╛

```

We will now test the convergence of some Markov chains. Remember from the lectures that chains were classified according to two properties:

1. Reducibility. An irreducible (or regular) chain has the property that every state is reachable from every state. Therefore the chain has a single stationary distribution. A reducible chain has not such property and, therefore, can have multiple stationary distributions. The general idea is that, if a chain has, say, two disconnected sets of states **A** and **B** and we start in a state $a \in \textbf{A}$, then we will never reach a state in **B**. In this case, the stationary distribution will depend on the transition probabilities of the states in **A**. The same occurs if we start in a state $b \in \textbf{B}$.

2. Periodicity. An irreducible chain is not guaranteed to convergence. To ensure convergence, we need an additional property: aperiodicity. An aperiodic chain avoids alternating forever between states without ever settling in a stationary distribution. A practical issue in that, although irreducible aperiodic chains are guaranteed to converge to a single stationary distribution, the convergence can be very slow, depending on the transition probabilities.

### Exercise

Let's implement an additional method `forwardUntilConvergence` that runs until convergence or a maximum number of iterations. We will establish convergence when the error between two consecutive state probability distributions is smaller than a threshold.

Let's start with the implementation of `forwardUntilConvergence`. We implemented a helper function `factorError` that computes the absolute error between two factors. The error can be throught of as a measure of distance between two factors.

In [ ]:
def factorError(f1, f2):
    """
    argument 
    `f1`, factor with the current state probability distribution in the chain.
    `f2`, factor with the previous state probability distribution in the chain.
    
    Returns absolute error between f1 and f2. 
    """
    assert f1.domain == f2.domain
    return np.sum(np.abs(f1.table - f2.table))

class MarkovModel(MarkovModel):
    def forwardUntilConvergence(self, max_iters=1000, eps=0.000001):
        '''
        Arguments:
        `n`, maximum number of time updates.
        `eps`, error threshold to determine convergence.
        Returns:
        A history of error values
        A factor that represents the current state of the chain after n time steps or the convergence error is less than eps.
        '''
        errors = []
        prevState = self.state
        for i in range(max_iters):
            newState = ... # TODO forward
            ... # TODO calculate error
            ... # TODO break loop if error is small
            ... # TODO append error to errors list
            prevState = newState
        return errors, newState

##################
# Test code
        
variable_remap = {
    "Weather_next": "Weather"
}
mm = MarkovModel(weatherStart, weatherTransition, variable_remap)
errors, state = mm.forwardUntilConvergence()
print(state)
plt.plot(errors, 'ro')
plt.show()


If you implemented your code correctly, you should see an output like this:

```
╒═══════════╤══════════╕
│ Weather   │       Pr │
╞═══════════╪══════════╡
│ sun       │ 0.750001 │
├───────────┼──────────┤
│ rain      │ 0.249999 │
╘═══════════╧══════════╛
```

### Exercise

Now, we can test the convergence of some Markov chains of the previous tutorial. 

The first one has the following transition matrix:

$\begin{bmatrix}
0 & 1 & 0 & 0 \\
1/2 & 0 & 1/2 & 0 \\
0 & 1/2 & 0 & 1/2 \\
0 & 0 & 1 & 0 \\
\end{bmatrix}$
    
This chain is irreducible since we can reach any state from any state. However, it is periodic since if we start from state 1 in time 1, we reach states 1 and 3 at odd times and 2 and 4 at even times. Its stationary distribution is $(1/6, 2/6, 2/6, 1/6)$.

Use the implemented function to confirm that this chain does not converge. Use the next cell to define the necessary variables and call the `forwardUntilConvergence` function.

In [ ]:
outcomeSpace_c1 = {
    'R':(0,1,2,3),
    'R_next':(0,1,2,3),
}


t = Factor(('R', 'R_next'), outcomeSpace_c1)
t[0, 0] = 0.0
t[0, 1] = 1.0
t[0, 2] = 0.0
t[0, 3] = 0.0
t[1, 0] = 0.5
t[1, 1] = 0.0
t[1, 2] = 0.5
t[1, 3] = 0.0
... # TODO finish transition factor


s = Factor(('R',), outcomeSpace_c1)
s[0] = 1.0
s[1] = 0.0
s[2] = 0.0
s[3] = 0.0


##################
# Test code

variable_remap = {
    "R_next": "R"
}
mm = MarkovModel(s, t, variable_remap)
errors, state = mm.forwardUntilConvergence()
print(state)
plt.plot(errors, 'ro')
plt.show()

The second one has the following transition matrix:

$\begin{bmatrix}
1/2 & 1/2 & 0 & 0 \\
1/2 & 0 & 1/2 & 0 \\
0 & 1/2 & 0 & 1/2 \\
0 & 0 & 0 & 1 \\
\end{bmatrix}$
    
This chain is reducible since state 4 is absorbing (once entered, cannot be left) and aperiodic. Its stationary distribution is $(0, 0, 0, 1)$.

Use the implemented function to verify this chain convergence. Use the next cell to define the necessary variables and call `forwardUntilConvergence` function.

In [ ]:
outcomeSpace_c2 = {
    'R':(0,1,2,3),
    'R_next':(0,1,2,3),
}

t = Factor(('R', 'R_next'), outcomeSpace_c2)
# We can set the values in the factor in a different way by directly setting the table
# This is quicker, but you have to be careful that the domain is in the right order
t.table = np.array(
    [
     [.5, .5, 0, 0],
    ... # TODO finish array
    ]
)

s = Factor(('R',), outcomeSpace_c2)
s.table = np.array(
    [1,0,0,0]
)

##################
# Test code

variable_remap = {
    "R_next": "R"
}
mm = MarkovModel(s, t, variable_remap)
errors, state = mm.forwardUntilConvergence()
print(state)
plt.plot(errors, 'ro')
plt.show()

The third one has the following transition matrix:

$\begin{bmatrix}
1/2 & 1/2 & 0 & 0 \\
1/2 & 1/2 & 0 & 0 \\
0 & 0 & 1/2 & 1/2 \\
0 & 0 & 1/2 & 1/2 \\
\end{bmatrix}$
    
This chain is reducible since we cannot reach states {3,4} from states {1,2} and vice-versa and aperiodic. It has two stationary distributions: $(1/2, 1/2, 0, 0)$ and $(0, 0, 1/2, 1/2)$.

Use the implemented function to verify this chain convergence. Use the next cell to define the necessary variables and call `forwardUntilConvergence` function.

In [ ]:
outcomeSpace_c3 = {
    'R':(0,1,2,3),
    'R_next':(0,1,2,3),
}

t = Factor(('R', 'R_next'), outcomeSpace_c3)
t.table = np.array(
    [[.5, .5, 0, 0],
     [.5, .5, 0, 0],
    ... # TODO
    ]
)

s = Factor(('R',), outcomeSpace_c3)
s.table = np.array(
    [1,0,0,0]
)

##################
# Test code

variable_remap = {
    "R_next": "R"
}
mm = MarkovModel(s, t, variable_remap)
errors, state = mm.forwardUntilConvergence()
print(state)
plt.plot(errors, 'ro')
plt.show()

Finally, we have a chain in the form:

$\begin{bmatrix}
1-e & e \\
e & 1-e \\
\end{bmatrix}$
    
This chain is irreducible and aperiodic, but its convergence can be very slow for small values of $e$. It has one stationary distribution: $(1/2, 1/2)$.

Use the implemented function to verify the chain convergence. Change the value of $e$. Use the next cell to define the necessary variables and call `forwardUntilConvergence` function.

In [ ]:
e = 0.0001

outcomeSpace_c4 = {
    'R':(0,1),
    'R_next':(0,1),
}

t = Factor(('R', 'R_next'), outcomeSpace_c4)
t.table = ... # TODO

s = Factor(('R',), outcomeSpace_c4)
s.table = np.array(
    [1,0]
)

##################
# Test code

variable_remap = {
    "R_next": "R"
}
mm = MarkovModel(s, t, variable_remap)
errors, state = mm.forwardUntilConvergence()
print(state)
plt.plot(errors, 'ro')
plt.show()


# PageRank algorithm

The PageRank algorithm is the original algorithm used by Google 1.0. It can be interpreted as a direct application of Markov chains. 

PageRank models the web as a state graph: pages are states and hyperlinks are transitions. The transition probabilities as set as follows:

Imagine you are surfing the web. 
* With 85% probability, you will jump to the next website by randomly selecting from the links on that page. 
* With 15% probability, you will jump randomly to any random page on the graph. 
* If you are at a page with no links, you will always jump to a random page on the graph.

Using these transition probabilities, the PageRank algorithm estimates the stationary distribution over webpages, and uses the probabilities of this distribution as an estimate of the "popularity" of every webpage.

The following figure from [Wikipedia](https://en.wikipedia.org/wiki/PageRank) illustrates the final probabilities (in percentages) of PageRank for a small graph.

![](https://raw.githubusercontent.com/UNSW-COMP9418/Week05/main/img/page_rank.png)

Let's use the mini-forward algorithm to implement the PageRank algorithm. We will use this figure to compare our results. We start by defining this graph. We will do it for you.

In [ ]:
# alpha sets the weight portion reserved for regular edges
alpha = .85

outcomePageRank = {
    "P":('A','B','C','D','E','F','G','H','I','J','K'),
    "P_next":('A','B','C','D','E','F','G','H','I','J','K'),
}

page_graph = Graph({
    "A" : [],
    'B' : ['C'],
    'C' : ['B'],
    'D' : ['B','A'],
    'E' : ['B','D','F'],
    'F' : ['B','E'],
    'G' : ['B','E'],
    'H' : ['B','E'],
    'I' : ['B','E'],
    'J' : ['E'],
    'K' : ['E'],
})

## Exercise

Fill in the function below to create a transition table according to the rules above. 

In [ ]:
def createTransitionTable(graph, outcomePageRank, alpha=0.85):
    N = len(graph)
    phantomTransition = Factor(('P', 'P_next'), outcomePageRank)
    
    # for every pair of nodes in the graph
    for u in outcomePageRank['P']:
        for v in outcomePageRank['P_next']:
            # find the number of outgoing links
            out_degree = len(graph.children(u))
            if out_degree == 0:
                # starting at a node 'u' with no neighbors, what's the probability of jumping to v?
                prob_v_given_u = ... # TODO
            else:
                if v in graph.children(u):
                    # starting at 'u', the probability of jumping to a connected node 'v'. 
                    # (Make sure you take into account the 15% possibility of randomly jumping to any node)
                    prob_v_given_u = ... # TODO
                else:
                    # starting at 'u', the probability of jumping to a random node 'v'
                    prob_v_given_u = ... # TODO 
            phantomTransition[u,v] = prob_v_given_u
    return phantomTransition
            
pageRankTransition = createTransitionTable(page_graph, outcomePageRank)
print(pageRankTransition)

Let's use ``GraphViz`` again to plot this graph and ensure we have designed it correctly. Note that we will not print all $11^2$ conditional probabilities, only the ones that correspond to following a link in the above graph. For a node with one outgoing edge, that edge should have a probability of $0.85 + 0.15\times(1/11) \approx 0.864 $, which is the sum of the probability of following the link $85$% and the probability of randomly choosing the connected node $15\%\times(1/11)$.

In [ ]:
dot = Digraph(engine="neato", comment='PageRank Markov chain')
dot.attr(overlap="false", splines="false", strict="true")

pos = {
    'A': '0,2!',
    'B': '2,2!',
    'C': '4,2!',
    'D': '0,1!',
    'E': '3,1!',
    'F': '4,1!',
    'G': '0,0!',
    'H': '1,0!',
    'I': '2,0!',
    'J': '3,0!',
    'K': '4,0!',
}

for v in page_graph:
    dot.node(v, pos=pos[v])

for v in page_graph:
    for w in page_graph.children(v):
        dot.edge(v, w, str(round(pageRankTransition[v,w],3)))

dot

### Running PageRank

In [ ]:
variable_remap = {
    "P_next": "P"
}

# We set all states with equal initial probabilities
startPageRank = Factor(('P',), outcomePageRank)
startPageRank['A'] = 1/11
startPageRank['B'] = 1/11
startPageRank['C'] = 1/11
startPageRank['D'] = 1/11
startPageRank['E'] = 1/11
startPageRank['F'] = 1/11
startPageRank['G'] = 1/11
startPageRank['H'] = 1/11
startPageRank['I'] = 1/11
startPageRank['J'] = 1/11
startPageRank['K'] = 1/11

pageRankTransition = createTransitionTable(page_graph, outcomePageRank)

mm = MarkovModel(startPageRank, pageRankTransition, variable_remap)
errors, page_rank = mm.forwardUntilConvergence()
print(page_rank)
plt.plot(errors, 'ro')
plt.show()

We can use ``GraphViz`` to visualise the results of the PageRank algorithm implementation. The next cell tries to mimic the Wikipedia figure.

In [ ]:
dot = Digraph(engine="neato", comment='PageRank Markov chain')
dot.attr(overlap="false", splines="false", strict="true")

pos = {
    'A': '0,2!',
    'B': '2,2!',
    'C': '4,2!',
    'D': '0,1!',
    'E': '3,1!',
    'F': '4,1!',
    'G': '0,0!',
    'H': '1,0!',
    'I': '2,0!',
    'J': '3,0!',
    'K': '4,0!',
}

for v in page_graph:
    dot.node(v, pos=pos[v], label=v+'\n'+str(round(page_rank[v]*100,1)))

for v in page_graph:
    for w in page_graph.children(v):
        dot.edge(v, w)

dot

![](https://raw.githubusercontent.com/UNSW-COMP9418/Week05/main/img/page_rank.png)

## Hidden Markov Models (HMMs)

We now turn our attention to HMMs. These are popular dynamic Bayesian Networks. We will implement the forward and Viterbi algorithms and verify how these algorithms provide answers to the questions on the theory part of this tutorial.

We start with the forward algorithm. According to the lecture slides, the forward algorithm for HMMs has the following form:

1. The transition step is similar to Markov chains. In this step, we have a passage of time and the distribution moves towards the stationary distribution.

    $P(x_t|e_{1:t-1}) = \sum_{x_{t-1}}P(x_{t-1}|e_{1:t-1})P(x_t|x_{t-1})$

    where, $P(x_t|e_{1:t-1})$ is the current state before observing the evidence $e_t$. $P(x_{t-1}|e_{1:t-1})$ is the previous state and $P(x_t|x_{t-1})$ is the transition probability.


2. The emission step has the following form:

    $P(x_t|e_{1:t}) \propto P(x_t|e_{1:t-1})P(e_t|x_t)$

    where, $P(x_t|e_{1:t-1})$ is the current state after observing the evidence $e_t$. $P(x_t|e_{1:t-1})$ is the current state before observing de evidence (obtained in the previous step) and $P(e_t|x_t)$ is the emission probability.


The symbol $\propto$ means that the emission step requires a normalization. The normalization is necessary because we are omitting the denominator in the emission update. The denominator is the probability of the evidence, which we frequently do not have readily available. Although we can compute such quantity, the normalization is usually easier to calculate.

### Exercise

Let's implement the forward algorithm for HMMs. 

To make this function more flexible, we will allow the user to pass an empty emission value. This means that no evidence was observed in this step. Also, the renormalization will be optional, so we will keep it commented for now.

In [ ]:
class HiddenMarkovModel():
    def __init__(self, start_state, transition, emission, variable_remap):
        '''
        Takes 3 arguments:
        - start_state: a factor representing the start state. E.g. domain might be ('A', 'B', 'C')
        - transition: a factor that represents the transition probs. E.g. P('A_next', 'B_next', 'C_next' | 'A', 'B', 'C')
        - emission: emission probabilities. E.g. P('O' | 'A', 'B', 'C')
        - variable_remap: a dictionary that maps new variable names to old variable names,
                            to reset the state after transition. E.g. {'A_next':'A', 'B_next':'B', 'C_next':'C'}
        '''
        self.state = start_state
        self.transition = transition
        self.emission = emission
        self.remap = variable_remap

        # These lists will be used later to find the mostly likely sequence of states
        self.history = []
        self.prev_history = []

    def forward(self, **emission_evi):
        # get state vars (to be marginalized later)
        state_vars = self.state.domain

        # join with transition factor
        f = ... # TODO

        # marginalize out old state vars, leaving only new state vars
        for var in state_vars:
            f = ... # TODO

        # remap variables to their original names
        f.domain = tuple(self.remap[var] for var in f.domain)
        self.state = f

        # set emission evidence
        emissionFactor = ... # TODO

        # join with state factor
        f = ... # TODO

        # marginalize out emission vars
        for var in f.domain:
            if var not in state_vars:
                f = ... # TODO
        self.state = f

        # normalize state (keep commented out for now)
        # self.state = self.state.normalize()

        return self.state

Let's use the weather example from the lecture to test our implementation. In this example, a graduate student tries to figure out the current weather state by observing this advisor carrying an umbrella.

These are the transition and emission probabilities:

In [ ]:
##################
# Test code

from collections import OrderedDict as odict

# possible outcomes, by variable
outcomeSpace = {
    'Weather':('sun','rain'),
    'Weather_next':('sun','rain'),
    'Umbrella': ('umbrella', 'no_umbrella'),
}

weatherStart = Factor(('Weather',), outcomeSpace)
weatherStart['sun'] = 0.5
weatherStart['rain'] = 0.5
weatherTransition = Factor(('Weather', 'Weather_next'), outcomeSpace)
weatherTransition['sun', 'sun'] = 0.7
weatherTransition['sun', 'rain'] = 0.3
weatherTransition['rain', 'sun'] = 0.3
weatherTransition['rain', 'rain'] = 0.7
weatherEmission = Factor(('Weather', 'Umbrella'), outcomeSpace)
weatherEmission['sun', 'umbrella'] = 0.2
weatherEmission['sun', 'no_umbrella'] = 0.8
weatherEmission['rain', 'umbrella'] = 0.9
weatherEmission['rain', 'no_umbrella'] = 0.1

variable_remap = {
    'Weather_next': 'Weather'
}

hmm = HiddenMarkovModel(weatherStart, weatherTransition, weatherEmission, variable_remap)
print(hmm.forward(Umbrella='umbrella'))

hmm = HiddenMarkovModel(weatherStart, weatherTransition, weatherEmission, variable_remap)
print(hmm.forward())

If you implemented your code correctly, you should see an output like this:

```
╒═══════════╤══════╕
│ Weather   │   Pr │
╞═══════════╪══════╡
│ sun       │ 0.1  │
├───────────┼──────┤
│ rain      │ 0.45 │
╘═══════════╧══════╛

╒═══════════╤══════╕
│ Weather   │   Pr │
╞═══════════╪══════╡
│ sun       │  0.5 │
├───────────┼──────┤
│ rain      │  0.5 │
╘═══════════╧══════╛
```

### Exercise

Let's now implement the batch version of the forward algorithm. The batch version takes as input a sequence of `n` observations and outputs an array of length `n` with the state distribution at each time step.

We have created a stub for you. You should pass as argument a list of evidence observations. Call the online version of the algorithm as many times as you have items in the emissionEviList.

In [ ]:
class HiddenMarkovModel(HiddenMarkovModel):
    def forwardBatch(self, n, **emission_evi):
        '''
        emission_evi: A dictionary of lists, each list containing the evidence list for a variable. 
                         Use `None` if no evidence for that timestep
        '''
        history = []
        for i in range(n):
            # select evidence for this timestep
            evi_dict = dict([(key, value[i]) for key, value in emission_evi.items() if value[i] is not None])
            
            # take a step forward
            state = ... # TODO
            history.append(state)
        return history

In [ ]:
hmm = HiddenMarkovModel(weatherStart, weatherTransition, weatherEmission, variable_remap)
timeLine = hmm.forwardBatch(2, Umbrella=['umbrella', 'umbrella'])
for t in range(len(timeLine)):
    print("Time: ", t)
    print(timeLine[t])
    print()

If your implementation is correct. You should see an output like this one:

```
Time:  0
╒═══════════╤══════╕
│ Weather   │   Pr │
╞═══════════╪══════╡
│ sun       │ 0.1  │
├───────────┼──────┤
│ rain      │ 0.45 │
╘═══════════╧══════╛


Time:  1
╒═══════════╤════════╕
│ Weather   │     Pr │
╞═══════════╪════════╡
│ sun       │ 0.041  │
├───────────┼────────┤
│ rain      │ 0.3105 │
╘═══════════╧════════╛

```

Now, let's implement the Viterbi algorithm. The Viterbi algorithm provides answers to queries in the form of the most likely explanation (MLE). In other words, the output will be the most likely instantiation for each of the hidden states.

According to the course slides, the Viterbi algorithm has the following equations:

$m_t[x_t] = P(e_t|x_t) max_{x_{t-1}}P(x_t|x_{t-1})m_{t-1}[x_{t-1}]$

where, 

- $m_t[x_t]$ is the MLE for time $t$.
- $P(e_t|x_t)$ is the emission probability.
- $P(x_t|x_{t-1})$ is the transition probability.
- $m_{t-1}[x_{t-1}]$ is the MLE for time $t-1$.

### Exercise

To implement the Viterbi algorithm, we will need an intermediate operation `maximize`. Such an operation is similar to `marginalize` in the sense that it will eliminate one variable of a factor. However, instead of summing out this variable, we will compute the maximum among the entries.

We have created a stub for you. You will need to complete a few gaps.

In [ ]:
import copy
class Factor(Factor):
    def maximize(self, var, return_prev=False):
        '''
        Usage: f.maximize('B'), where 'B' is a variable name.
        This function removes a variable from the domain, and maximizes over that variable in the table
        The return_prev argument will be used when tracing back the viterbi algorithm
        '''
        
        # create new domain
        ... # TODO
        
        # remove an axis of the table by taking a maximum over that axis
        axis = self.domain.index(var)
        new_table = ... # TODO
        
        # create a new factor to be returned
        outputFactor = self.__class__(tuple(new_dom),self.outcomeSpace, new_table)

        if return_prev:
            # get the index chosen when taking maximum (to be used later in the viterbi algorithm)
            prev = np.argmax(self.table, axis=axis)
            return outputFactor, prev
        else:
            return outputFactor


################
# Test code

# re-initialize the factors so that it has access to the maximize function
weatherStart = Factor(weatherStart.domain, weatherStart.outcomeSpace, weatherStart.table)
weatherTransition = Factor(weatherTransition.domain, weatherTransition.outcomeSpace, weatherTransition.table)
weatherEmission = Factor(weatherEmission.domain, weatherEmission.outcomeSpace, weatherEmission.table)

print(weatherEmission.maximize('Umbrella'))

If your implementation is correct, you should see the following output:

```
╒═══════════╤══════╕
│ Weather   │   Pr │
╞═══════════╪══════╡
│ sun       │  0.8 │
├───────────┼──────┤
│ rain      │  0.9 │
╘═══════════╧══════╛
```

### Exercise

Now we are in the position to implement the Viterbi algorithm. We have started the implementation and left a few details for you to fill in. As before, we will code the online version first. The online version provides the output for a single time and evidence observation increment. The batch version is a simple extension of the online algorithm.

In [ ]:
class HiddenMarkovModel(HiddenMarkovModel):
    def viterbi(self, **emission_evi):
        '''
        This function is very similar to the forward algorithm. 
        For simplicity, we will assume that there is only one state variable, and one emission variable.
        '''

        # confirm that state and emission each have 1 variable 
        assert len(self.state.domain) == 1
        assert len(self.emission.domain) == 2
        assert len(self.transition.domain) == 2

        # get state and evidence var names (to be marginalized and maximised out later)
        state_var_name = self.state.domain[0]
        emission_vars = [v for v in self.emission.domain if v not in self.state.domain]
        emission_var_name = emission_vars[0]

        # join with transition factor
        f = ... # TODO

        # maximize out old state vars, leaving only new state vars
        f, prev = ... # TODO (use return_prev to also return prev)
        self.prev_history.append(prev) # save prev for use in traceback

        # remap variables to their original names
        f.domain = tuple(self.remap[var] for var in f.domain)
        self.state = f

        # set emission evidence
        emissionFactor = ... # TODO

        # join with state factor
        f = ... # TODO

        # marginalize out emission vars
        f = ... # TODO
        self.state = f

        # normalize state (keep commented out for now)
        # self.state = self.state.normalize()

        self.history.append(self.state)

        return self.state

    def viterbiBatch(self, n,  **emission_evi):
        '''
        emission_evi: A dictionary of lists, each list containing the evidence list for a variable. 
                         Use `None` if no evidence for that timestep
        '''
        for i in range(n):
            # get evidence for this timestep
            evi_dict = dict([(key, value[i]) for key, value in emission_evi.items() if value[i] is not None])
            ... # TODO take a step using the `viterbi` method
        return self.history

##################
# Test code

hmm = HiddenMarkovModel(weatherStart, weatherTransition, weatherEmission, variable_remap)
print(hmm.viterbi(Umbrella='umbrella'))


If your implementation is correct, you should see the following output:

```
╒═══════════╤═══════╕
│ Weather   │    Pr │
╞═══════════╪═══════╡
│ sun       │ 0.07  │
├───────────┼───────┤
│ rain      │ 0.315 │
╘═══════════╧═══════╛
```

Now, we can test the batch version of the Viterbi algorithm with a simple code that runs 2 iterations and prints the timeline.

In [ ]:
hmm = HiddenMarkovModel(weatherStart, weatherTransition, weatherEmission, variable_remap)
timeLine = hmm.viterbiBatch(2, Umbrella=['umbrella', 'umbrella'])
for t in range(len(timeLine)):
    print("Time: ", t)
    print(timeLine[t])

If your implementation is correct, you should see the following output:

```
Time:  0
╒═══════════╤═══════╕
│ Weather   │    Pr │
╞═══════════╪═══════╡
│ sun       │ 0.07  │
├───────────┼───────┤
│ rain      │ 0.315 │
╘═══════════╧═══════╛

Time:  1
╒═══════════╤═════════╕
│ Weather   │      Pr │
╞═══════════╪═════════╡
│ sun       │ 0.0189  │
├───────────┼─────────┤
│ rain      │ 0.19845 │
╘═══════════╧═════════╛
```

Before we conclude with an exercise, here are some suggestions of ways to improve the source code of this tutorial:

1. The forward algorithm typically does not normalize the intermediate results. You can make this step optional with a flag. This change can also be done to the Viterbi algorithm.

2. If you do not normalize, the probabilities will assume small values due to the sequence of multiplications. In this case, operating with log-probabilities will decrease the chance of having underflows.

3. For simplicity, we are assuming the update steps are composed by a transition followed by emission. But it is not always the case. It would be better if these steps can be inverted or, even better, implemented independently.

## Finding the MPE Assignment

As we discussed in the lecture, we can find the MPE assigment from the output computed by the Viterbi algorithm. Remember, that the correct assignment is obtained when we trace back the computations, starting with the last state and move towards the first state. There are two main approaches for finding the MPE assignment:

1. Use an additional data structure to store pointers indicating the path of the highest probability.

2. Use the output of the Viterbi algorithm and trace back the computations.

In the next cell, we provide a function to find the MPE assignment using the output of the ``viterbi`` function. We will use the first method of finding the MPE assignment.

In [ ]:
class HiddenMarkovModel(HiddenMarkovModel):
    def traceBack(self):
        '''
        This function iterates backwards over the history to find the most 
        likely sequence of states.
        For simplicity, this function assumes there is one state variable
        '''
        # get most likely outcome of final state
        index = np.argmax(self.history[-1].table)
        
        # Go through "prev_history" in reverse
        indexList = []
        for prev in reversed(self.prev_history):
            indexList.append(index)
            index = prev[index]
        indexList = reversed(indexList)

        # translate the indicies into the outcomes they represent
        mleList = []
        stateVar = self.state.domain[0]
        for idx in indexList:
            mleList.append(self.state.outcomeSpace[stateVar][idx]) 
        return mleList

####################
## Test code

hmm = HiddenMarkovModel(weatherStart, weatherTransition, weatherEmission, variable_remap)
timeLine = hmm.viterbiBatch(2, Umbrella=['umbrella', 'umbrella'])
mpe = hmm.traceBack()
print(mpe)

### Exercise

You can use the implemented code to find the numerical answers to the questions of the theory part of this tutorial.

### Question 1

Lisa is given a fair coin $C_1$ and asked to flip it eight times in a row. Lisa also has a biased coin $C_2$ with probability 0.8 of landing heads. All we know is that Lisa flipped the fair coin initially but we believe that she intends to switch to the biased coin and that she tends to be 10% successful in performing the switch. Suppose that we observe the outcome of the eight coin flips and want to find out whether Lisa managed to perform a coin switch and when. What is the solution to this problem assuming that the flips came out as follows:

    a. tails, tails, tails, heads, heads, heads, heads, heads
    b. tails, tails, heads, heads, heads, heads, heads, heads

### Question 2

Consider a cow that may be infected with a disease that can possibly be detected by performing a milk test. The test is performed on five consecutive days, leading to five outcomes. We want to determine the state of the cow's infection over these days given the test outcomes. The prior probability of an infection on day one is 1/10,000; the test false positive rate is 5/1,000; and its false negative rate is 1/1,000. Moreover, the state of infection at a given day depends only on its state at the previous day. In particular, the probability of a new infection on a given day is 2/10,000, while the probability that an infection would persist to the next day os 7/10.

What is the most likely state of the cow's infection over the five days given the following test outcomes:

    a. positive, positive, negative, positive, positive
    b. positive, negative, negative, positive, positive
    c. positive, nagative, negative, negative, positive

## Final Task
Add the `maximize` method to your DiscreteFactors.py file